<a href="https://colab.research.google.com/github/josh130588/MLAIMAR2024/blob/main/CaseStudy03_ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Upload the Dataset
from google.colab import files

# Upload the dataset zip file
uploaded = files.upload()

# List the uploaded files to confirm
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

# Step 2: Unzip the Dataset
import zipfile
import os

# Unzip the uploaded file
zip_file_name = 'fruits.zip'  # Change this if the uploaded file has a different name

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('fruits')

# Check the extracted files
os.listdir('fruits')

# Step 3: Adjust Paths in Code
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

# Define paths to dataset
train_dir = os.path.join('fruits', 'Training')
test_dir = os.path.join('fruits', 'Test')

# Verify that the directories exist
assert os.path.exists(train_dir), f"Training directory {train_dir} does not exist"
assert os.path.exists(test_dir), f"Test directory {test_dir} does not exist"

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Building the CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Evaluating the Model
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc}')

# Saving the Model
model.save('fruits_vegetables_classifier.h5')